In [251]:
import paho.mqtt.client as mqtt
import json
import time

In [252]:
import paho.mqtt.client as mqtt
import json
import time

MQTT_BROKER = "mqtt-dashboard.com"
MQTT_PORT = 1883
MQTT_TOPIC_PID = "robot/set/pid"
MQTT_TOPIC_MOTORS = "robot/set/motors"
MQTT_TOPIC_LOG = "robot/pid/log"
MQTT_TOPIC_RESPONSE = "robot/request"

In [253]:
mqtt_client = mqtt.Client("PID ctl")

logs = []

In [254]:
def send_mqtt_message(topic, message):
    mqtt_client.publish(topic, json.dumps(message))

In [255]:
def run_experiment(Kp, Ti, Td):
    global logs
    global mqtt_client
    
    logs = []
    mqtt_client = mqtt.Client("PID ctl")
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
    mqtt_client.subscribe(MQTT_TOPIC_LOG)
    mqtt_client.on_message = on_log_message
    
    pid_settings = {"Kp": Kp, "Ti": Ti, "Td": Td}
    send_mqtt_message(MQTT_TOPIC_PID, pid_settings)

    motors_speed = {"left": 70, "right": 70}
    send_mqtt_message(MQTT_TOPIC_MOTORS, motors_speed)

    start_time = time.time()
    while time.time() - start_time < 3:
        mqtt_client.loop()

    motors_speed = {"left": 0, "right": 0}
    send_mqtt_message(MQTT_TOPIC_MOTORS, motors_speed)

    time.sleep(2)
    
    send_mqtt_message(MQTT_TOPIC_RESPONSE, "")

def on_log_message(client, userdata, message):
    log_data = json.loads(message.payload.decode())
    logs.append(log_data)  

In [256]:
mqtt_client.subscribe(MQTT_TOPIC_LOG)
mqtt_client.on_message = on_log_message

In [257]:
import math
def calc_error(Kp, Ti, Td):
    global logs

    absolute_errors = []
    relative_errors = []
    settling_times = []

    sampling_time = 5

    run_experiment(Kp, Ti, Td)

    for i in range(len(logs)):
        measurement = logs[i]
        absolute_error = abs(measurement['target'] - measurement['actual'])
        relative_error = (absolute_error / measurement['target']) * 100 if measurement['target'] != 0 else 0
        absolute_errors.append(absolute_error)
        relative_errors.append(relative_error)
        
        # Sprawdzenie, czy wartość docelowa została osiągnięta
        if relative_error < 2:  # Możesz dostosować wartość graniczną według potrzeb
            settling_time = i * sampling_time
            settling_times.append(settling_time)

    mean_error = sum(absolute_errors) / len(absolute_errors)

    mean_relative_error = sum(relative_errors) / len(relative_errors)

    rmse_error = math.sqrt(sum([(error ** 2) for error in absolute_errors]) / len(absolute_errors))

    average_settling_time = sum(settling_times) / len(settling_times) if settling_times else None

    # Wyświetlenie wyników
    print("Mean error: ", mean_error)
    print("Mean relative error: ", mean_relative_error)
    print("(RMSE): ", rmse_error)
    print("Average settling time: ", average_settling_time, " ms")

    print(logs)

In [258]:
PID_settings = [
    { "Kp" : 22.75297759, "Ti" : 41.13985303, "Td" : 43.13750817 },
    { "Kp" : 43.8126484, "Ti" : 6.42690804, "Td" : 47.58715438 },
    { "Kp" : 40.94357286, "Ti" : 28.49773496, "Td" : 43.95695723 }
]

# for setting in PID_settings:
#     print(setting)
#     calc_error(setting["Kp"], setting["Ti"], setting["Td"])

In [259]:
setting = PID_settings[0]
calc_error(setting["Kp"], setting["Ti"], setting["Td"])

Mean error:  0.5050366300366301
Mean relative error:  13.254698548328463
(RMSE):  0.6251749938167495
Average settling time:  1210.2777777777778  ms
[{'id': 0, 'target': 0.0, 'actual': 0.39}, {'id': 1, 'target': 0.0, 'actual': 0.27}, {'id': 2, 'target': 0.0, 'actual': 0.19}, {'id': 3, 'target': 0.0, 'actual': 0.13}, {'id': 4, 'target': 0.0, 'actual': 0.09}, {'id': 5, 'target': 0.0, 'actual': 0.07}, {'id': 6, 'target': 0.0, 'actual': 0.05}, {'id': 7, 'target': 0.0, 'actual': 0.03}, {'id': 8, 'target': 0.0, 'actual': 0.02}, {'id': 9, 'target': 0.0, 'actual': 0.02}, {'id': 10, 'target': 0.0, 'actual': 0.01}, {'id': 11, 'target': 0.0, 'actual': 0.01}, {'id': 12, 'target': 0.0, 'actual': 0.01}, {'id': 13, 'target': 0.0, 'actual': 0.0}, {'id': 14, 'target': 0.0, 'actual': 0.0}, {'id': 15, 'target': 0.0, 'actual': 0.0}, {'id': 16, 'target': 0.0, 'actual': 0.0}, {'id': 17, 'target': 0.0, 'actual': 0.0}, {'id': 18, 'target': 0.0, 'actual': 0.0}, {'id': 19, 'target': 0.0, 'actual': 0.0}, {'id': 2

In [260]:
# setting = PID_settings[1]
# calc_error(setting["Kp"], setting["Ti"], setting["Td"])